# TMA Analysis code supporting technote SITCOMTN-057
Craig Lage - 15-Nov-23  Updated 18-Dec-23 to use TMAEvents.

This notebook characterizes several things associated with the TMA: 

1. Slew and Settle times
2. Mount jitter


# Prepare the notebook

In [1]:
# Directory to store the data
from pathlib import Path
dataDir = Path("./plots")
dataDir.mkdir(exist_ok=True, parents=True)

# You can include a list of different days and blocks,
# including more than one block on the same day, if desired
# You can also create a loop to build this list.
dayBlockPairs = [[20231214, 146], [20231215, 146]]

# For the jitter tests, the parameters below allow you to add
# a delay after the start of the tracking event, or before the
# end of the tracking event
delayAfterStart = 0.0 # This will give some time to settle before evaluating the jitter
delayBeforeEnd = 1.0 # This is needed to prevent th start of the next slew from impacting the jitter

In [2]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.summit.utils import getCurrentDayObs_int, dayObsIntToString
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import getEfdData

In [6]:
client = EfdClient("idf_efd")
eventMaker = TMAEventMaker()

# These functions will be part of summit_utils once DM-42039 merges.  After that, we can just import them, but for now they are here.

In [19]:
def getAzimuthElevationDataForEvent(client, event, prePadding=0, postPadding=0):
    """Get the data for the az/el telemetry topics for a given TMAEvent.

    Parameters
    ----------
    client : `lsst_efd_client.efd_helper.EfdClient`
        The EFD client to use.
    event : `lsst.summit.utils.tmaUtils.TMAEvent`
        The event to get the data for.
    prePadding : `float`, optional
        The amount of time to pad the event with before the start time, in
        seconds.
    postPadding : `float`, optional
        The amount of time to pad the event with after the end time, in
        seconds.

    Returns
    -------
    azimuthData : `pd.DataFrame`
        The azimuth data for the specified event.
    elevationData : `pd.DataFrame`
        The elevation data for the specified event.
    """
    azimuthData = getEfdData(client,
                             'lsst.sal.MTMount.azimuth',
                             event=event,
                             prePadding=prePadding,
                             postPadding=postPadding)
    elevationData = getEfdData(client,
                               'lsst.sal.MTMount.elevation',
                               event=event,
                               prePadding=prePadding,
                               postPadding=postPadding)
    print(event.type.name, event.seqNum, azimuthData.columns)
    print(elevationData.columns)
    if event.type.name == 'TRACKING':
        # Need to pad this data for the interpolation to work right
        pointingData = getEfdData(client,
                                  'lsst.sal.MTPtg.currentTargetStatus',
                                  event=event,
                                  prePadding=1.0,
                                  postPadding=1.0)

        azTimes = azimuthData['timestamp'].values
        elTimes = elevationData['timestamp'].values
        ptgTimes = pointingData['timestamp'].values
        azValues = azimuthData['actualPosition'].values
        elValues = elevationData['actualPosition'].values
        # Need to interpolate because demand and actual data streams
        # have different lengths
        azDemandInterp = np.interp(azTimes, ptgTimes, pointingData['demandAz'])
        elDemandInterp = np.interp(elTimes, ptgTimes, pointingData['demandEl'])
        azError = (azValues - azDemandInterp) * 3600
        elError = (elValues - elDemandInterp) * 3600
        # Because of small timebase errors, there can be an offset in the
        # errors. I take this out by subtracting the median of the errors.
        # This is a fudge, but I think better than the polynomial fit.
        azError -= np.median(azError)
        elError -= np.median(elError)
        azimuthData['azError'] = azError
        elevationData['elError'] = elError

    return azimuthData, elevationData


def plotEvent(client, event, fig=None, prePadding=0, postPadding=0, commands={},
              azimuthData=None, elevationData=None):
    """Plot the TMA axis positions over the course of a given TMAEvent.

    Plots the axis motion profiles for the given event, with optional padding
    at the start and end of the event. If the data is provided via the
    azimuthData and elevationData parameters, it will be used, otherwise it
    will be queried from the EFD.

    Optionally plots any commands issued during or around the event, if these
    are supplied. Commands are supplied as a dictionary of the command topic
    strings, with values as astro.time.Time objects at which the command was
    issued.

    Parameters
    ----------
    client : `lsst_efd_client.efd_helper.EfdClient`
        The EFD client to use.
    event : `lsst.summit.utils.tmaUtils.TMAEvent`
        The event to plot.
    fig : `matplotlib.figure.Figure`, optional
        The figure to plot on. If not specified, a new figure will be created.
    prePadding : `float`, optional
        The amount of time to pad the event with before the start time, in
        seconds.
    postPadding : `float`, optional
        The amount of time to pad the event with after the end time, in
        seconds.
    commands : `dict` of `str` : `astropy.time.Time`, optional
        A dictionary of commands to plot on the figure. The keys are the topic
        names, and the values are the times at which the commands were sent.
    azimuthData : `pd.DataFrame`, optional
        The azimuth data to plot. If not specified, it will be queried from the
        EFD.
    elevationData : `pd.DataFrame`, optional
        The elevation data to plot. If not specified, it will be queried from
        the EFD.

    Returns
    -------
    fig : `matplotlib.figure.Figure`
        The figure on which the plot was made.
    """
    def tickFormatter(value, tick_number):
        # Convert the value to a string without subtracting large numbers
        # tick_number is unused.
        return f"{value:.2f}"

    # plot any commands we might have
    if not isinstance(commands, dict):
        raise TypeError('commands must be a dict of command names with values as'
                        ' astropy.time.Time values')

    if fig is None:
        fig = plt.figure(figsize=(10, 8))
        log = logging.getLogger(__name__)
        log.warning("Making new matplotlib figure - if this is in a loop you're going to have a bad time."
                    " Pass in a figure with fig = plt.figure(figsize=(10, 8)) to avoid this warning.")

    fig.clear()
    if event.type.name == 'TRACKING':
        ax1, ax1p5, ax2 = fig.subplots(3,
                                       sharex=True,
                                       gridspec_kw={'wspace': 0,
                                                    'hspace': 0,
                                                    'height_ratios': [2.5, 1, 1]})
    else:
        ax1, ax2 = fig.subplots(2,
                                sharex=True,
                                gridspec_kw={'wspace': 0,
                                             'hspace': 0,
                                             'height_ratios': [2.5, 1]})

    if azimuthData is None or elevationData is None:
        azimuthData, elevationData = getAzimuthElevationDataForEvent(client,
                                                                     event,
                                                                     prePadding=prePadding,
                                                                     postPadding=postPadding)

    # Use the native color cycle for the lines. Because they're on different
    # axes they don't cycle by themselves
    lineColors = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
    colorCounter = 0

    ax1.plot(azimuthData['actualPosition'], label='Azimuth position', c=lineColors[colorCounter])
    colorCounter += 1
    ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax1.set_ylabel('Azimuth (degrees)')

    ax1_twin = ax1.twinx()
    ax1_twin.plot(elevationData['actualPosition'], label='Elevation position', c=lineColors[colorCounter])
    colorCounter += 1
    ax1_twin.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax1_twin.set_ylabel('Elevation (degrees)')
    ax1.set_xticks([])  # remove x tick labels on the hidden upper x-axis

    ax2_twin = ax2.twinx()
    ax2.plot(azimuthData['actualTorque'], label='Azimuth torque', c=lineColors[colorCounter])
    colorCounter += 1
    ax2_twin.plot(elevationData['actualTorque'], label='Elevation torque', c=lineColors[colorCounter])
    colorCounter += 1
    ax2.set_ylabel('Azimuth torque (Nm)')
    ax2_twin.set_ylabel('Elevation torque (Nm)')
    ax2.set_xlabel('Time (UTC)')  # yes, it really is UTC, matplotlib converts this automatically!

    # put the ticks at an angle, and right align with the tick marks
    ax2.set_xticks(ax2.get_xticks())  # needed to supress a user warning
    xlabels = ax2.get_xticks()
    ax2.set_xticklabels(xlabels, rotation=40, ha='right')
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    if event.type.name == 'TRACKING':
        # Calculate RMS
        azValues = azimuthData['azError'].values
        elValues = elevationData['elError'].values
        azRms = np.sqrt(np.mean(azValues * azValues))
        elRms = np.sqrt(np.mean(elValues * elValues))

        # Calculate Image impact RMS
        # We are less sensitive to Az errors near the zenith
        imageAzRms = azRms * np.cos(elValues[0] * np.pi / 180.0)
        imageElRms = elRms

        ax1p5.plot(azimuthData['azError'], label='Azimuth error', c=lineColors[colorCounter])
        colorCounter += 1
        ax1p5.plot(elevationData['elError'], label='Elevation error', c=lineColors[colorCounter])
        colorCounter += 1
        ax1p5.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
        ax1p5.set_ylabel('Tracking error (arcsec)')
        ax1p5.set_xticks([])  # remove x tick labels on the hidden upper x-axis
        ax1p5.set_ylim(-0.5, 0.5)
        ax1p5.set_yticks([-0.25, 0.0, 0.25])
        ax1p5.legend()
        ax1p5.text(0.2, 0.9,
                   f'Az image RMS = {imageAzRms:.3f} arcsec,   El image RMS = {imageElRms:.3f} arcsec',
                   transform=ax1p5.transAxes)

    if prePadding or postPadding:
        # note the conversion to utc because the x-axis from the dataframe
        # already got automagically converted when plotting before, so this is
        # necessary for things to line up
        ax1_twin.axvline(event.begin.utc.datetime, c='k', ls='--', alpha=0.5, label='Event begin/end')
        ax1_twin.axvline(event.end.utc.datetime, c='k', ls='--', alpha=0.5)
        # extend lines down across lower plot, but do not re-add label
        ax2_twin.axvline(event.begin.utc.datetime, c='k', ls='--', alpha=0.5)
        ax2_twin.axvline(event.end.utc.datetime, c='k', ls='--', alpha=0.5)

    for command, commandTime in commands.items():
        # if commands weren't found, the item is set to None. This is common
        # for events so handle it gracefully and silently. The command finding
        # code logs about lack of commands found so no need to mention here.
        if commandTime is None:
            continue
        ax1_twin.axvline(commandTime.utc.datetime, c=lineColors[colorCounter],
                         ls='--', alpha=0.75, label=f'{command}')
        # extend lines down across lower plot, but do not re-add label
        ax2_twin.axvline(commandTime.utc.datetime, c=lineColors[colorCounter],
                         ls='--', alpha=0.75)
        colorCounter += 1

    # combine the legends and put inside the plot
    handles1a, labels1a = ax1.get_legend_handles_labels()
    handles1b, labels1b = ax1_twin.get_legend_handles_labels()
    handles2a, labels2a = ax2.get_legend_handles_labels()
    handles2b, labels2b = ax2_twin.get_legend_handles_labels()

    handles = handles1a + handles1b + handles2a + handles2b
    labels = labels1a + labels1b + labels2a + labels2b
    # ax2 is "in front" of ax1 because it has the vlines plotted on it, and
    # vlines are on ax2 so that they appear at the bottom of the legend, so
    # make sure to plot the legend on ax2, otherwise the vlines will go on top
    # of the otherwise-opaque legend.
    ax1_twin.legend(handles, labels, facecolor='white', framealpha=1)

    # Add title with the event name, type etc
    dayObsStr = dayObsIntToString(event.dayObs)
    title = (f"{dayObsStr} - seqNum {event.seqNum} (version {event.version})"  # top line, rest below
             f"\nDuration = {event.duration:.2f}s"
             f" Event type: {event.type.name}"
             f" End reason: {event.endReason.name}"
             )
    ax1_twin.set_title(title)
    if event.type.name == 'TRACKING':
        return fig, imageAzRms, imageElRms
    else:
        return fig




# Cycle through the events plotting the tracking errors, and getting the data for the slew time histogram and the jitter error histograms.

In [20]:
# These are for plotting the slew time distributions
slewTimes = []
slewDist = []

# These are for plotting the jitter distributions
azRmsVals = []
elRmsVals = []
imRmsVals = []

pdf = PdfPages(str(dataDir / "Mount_Jitter_Plots.pdf"))
fig = plt.figure(figsize=(10, 8))
for i, [dayObs, blockNum] in enumerate(dayBlockPairs):
    events = eventMaker.getEvents(dayObs)
    blockParser = BlockParser(dayObs)  # get the info for the day
    seqNums = blockParser.getSeqNums(blockNum)  # get the seqNums for the specified block

    for seqNum in seqNums[120:125]:
            #try:
            event = events[seqNum]
            if event.type.name == 'SLEWING':
                continue
                azimuthData, elevationData = getAzimuthElevationDataForEvent(
                    client, event, prePadding=0, postPadding=0)
                azValues = azimuthData['actualPosition'].values
                elValues = elevationData['actualPosition'].values
                azShift = abs(azValues[0] - azValues[-1])
                elShift = abs(elValues[0] - elValues[-1])
                azShiftMod = azShift * np.cos(elValues[0]*np.pi/180.0)
                shift = np.sqrt(elShift*elShift + azShiftMod*azShiftMod)
                if shift > 0.2 and shift < 10.0:
                    slewDist.append(shift)
                    slewTimes.append(event.duration)

            elif event.type.name == 'TRACKING':
                fig, imageAzRms, imageElRms = plotEvent(client, event, fig=fig, 
                        prePadding=delayAfterStart, postPadding=-delayBeforeEnd, commands={},
                        azimuthData=None, elevationData=None)
                azRmsVals.append(imageAzRms)
                elRmsVals.append(imageElRms)
                imRmsVals.append(np.sqrt(imageAzRms**2 + imageElRms**2))
                pdf.savefig(fig)  # saves the current figure into a pdf page
                plt.clf()
            else:
                continue
            #except:
            #continue
pdf.close()

TRACKING 123 Index(['actualPosition', 'actualTorque', 'actualVelocity', 'demandPosition',
       'demandVelocity', 'private_efdStamp', 'private_identity',
       'private_kafkaStamp', 'private_origin', 'private_rcvStamp',
       'private_revCode', 'private_seqNum', 'private_sndStamp', 'timestamp'],
      dtype='object')
Index(['actualPosition', 'actualTorque', 'actualVelocity', 'demandPosition',
       'demandVelocity', 'private_efdStamp', 'private_identity',
       'private_kafkaStamp', 'private_origin', 'private_rcvStamp',
       'private_revCode', 'private_seqNum', 'private_sndStamp', 'timestamp'],
      dtype='object')
TRACKING 125 Index(['actualPosition', 'actualTorque', 'actualVelocity', 'demandPosition',
       'demandVelocity', 'private_efdStamp', 'private_identity',
       'private_kafkaStamp', 'private_origin', 'private_rcvStamp',
       'private_revCode', 'private_seqNum', 'private_sndStamp', 'timestamp'],
      dtype='object')
Index(['actualPosition', 'actualTorque', 'actua

TRACKING 136 RangeIndex(start=0, stop=0, step=1)
RangeIndex(start=0, stop=0, step=1)


KeyError: 'timestamp'

In [ ]:
%matplotlib inline
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.hist(slewTimes, bins=50, range=(0,50))
plt.xlabel("Slew and settle time (seconds)")
plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slewDist, slewTimes)
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")
plt.plot([3.5,3.5],[0,10], ls='--', color='black')
plt.plot([0,10],[4.0,4.0], ls='--', color='black')
plt.xlim(0,10)
plt.ylim(0,10)
plt.savefig(str(dataDir / "Slew_Settle_Times.pdf"))

# This plots histograms of the tracking jitter.

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.suptitle("MT Mount RMS Jitter - 20220126", fontsize = 18)
azRmsVals = mount_data['azRmsVals']
elRmsVals = mount_data['elRmsVals']
imRmsVals = mount_data['imRmsVals']
azMed = np.median(azRmsVals)
elMed = np.median(elRmsVals)
imMed = np.median(imRmsVals)
plt.subplots_adjust(wspace=0.2)
plt.subplot(1,3,1)
plt.title(f"Azimuth RMS, N={len(azRmsVals)}")
plt.hist(azRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={azMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.subplot(1,3,2)
plt.title(f"Elevation RMS, N={len(azRmsVals)}")
plt.hist(elRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={elMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.subplot(1,3,3)
plt.title(f"Image Impact RMS, N={len(azRmsVals)}")
plt.hist(imRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={imMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.savefig(str(dataDir / "Jitter_Summary_Corrected.pdf"))

# This is just to plot an example of the slews and tracks.  It is not needed for the analysis.

In [15]:
%matplotlib inline
[dayObs, blockNum] = dayBlockPairs[0]
events = eventMaker.getEvents(dayObs)
blockParser = BlockParser(dayObs)  # get the info for the day
seqNums = blockParser.getSeqNums(blockNum)  # get the seqNums for the specified block
print(f"On {dayObs}, there are {len(seqNums)} events associated with block {blockNum}")

medianSeqNum = int((seqNums[0] + seqNums[-1]) / 2)
firstEvent = medianSeqNum - 4
lastEvent = medianSeqNum + 4
start = events[firstEvent].begin
end = events[lastEvent].end
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                        ['*'],  start, end)
plt.subplot(1,1,1)
plt.title(f"Azimuth Slew and Tracking {dayObs}")
ax1 = az['actualPosition'].plot(color='red')
for i in range(firstEvent, lastEvent+1):
    if events[i].type.name != 'SLEWING':
        continue
    ssTime = events[i].begin.isot  
    ax1.axvline(ss_time, color="black", linestyle="--")
    ipTime = events[i].end.isot  
    ax1.axvline(ip_time, color="blue", linestyle="--")
#ax1.set_xlim(start.isot, end.isot)
ax1.axvline(ssTime, color="black", linestyle="--", label="Start slew")
ax1.axvline(ipTime, color="blue", linestyle="--", label="InPosition")
ax1.set_ylabel("Azimuth(degrees)")
ax1.legend()
plt.savefig(str(dataDir / f"Slew_Track_Example_{dayObs}.pdf"))

On 20231214, there are 283 events associated with block 146


In [15]:
[dayObs, blockNum] = dayBlockPairs[0]
events = eventMaker.getEvents(dayObs)
blockParser = BlockParser(dayObs)  # get the info for the day
seqNums = blockParser.getSeqNums(blockNum)  # get the seqNums for the specified block
event = events[121]
fig, imageAzRms, imageElRms = plotEvent(client, event, fig=fig, 
                        prePadding=delayAfterStart, postPadding=-delayBeforeEnd, commands={},
                        azimuthData=None, elevationData=None)


Index(['actualPosition', 'actualTorque', 'actualVelocity', 'demandPosition',
       'demandVelocity', 'private_efdStamp', 'private_identity',
       'private_kafkaStamp', 'private_origin', 'private_rcvStamp',
       'private_revCode', 'private_seqNum', 'private_sndStamp', 'timestamp'],
      dtype='object')
Index(['actualPosition', 'actualTorque', 'actualVelocity', 'demandPosition',
       'demandVelocity', 'private_efdStamp', 'private_identity',
       'private_kafkaStamp', 'private_origin', 'private_rcvStamp',
       'private_revCode', 'private_seqNum', 'private_sndStamp', 'timestamp'],
      dtype='object')


In [16]:
event.type

TMAState.TRACKING